In [1]:
import sys
import os


from LSTMAutoEncoderModule import *

import torch
import torch.nn as nn
import pickle
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchsummary import summary
from torchmetrics.regression import R2Score, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = '../Data/'

electric_df = pd.read_csv(DATA_PATH + 'electric_df_clear_2.csv', index_col = 0)
water_df = pd.read_csv(DATA_PATH + 'water_diff_df_clear.csv')

In [3]:
print(electric_df.head())
print()
print(water_df.head())

               0     1     2     3     4     5     6     7     8     9  ...  \
Unnamed: 0                                                              ...   
0           1.23  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  ...   
1           1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  ...   
2           1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  ...   
3           1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  ...   
4           1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  1.39  ...   

              18    19    20    21    22    23    24    25    26    27  
Unnamed: 0                                                              
0           1.77  1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  
1           1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  
2           1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  
3           1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  
4       

In [4]:
EPOCH = 10000
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [5]:
electric_trainDS = CustomDataset(electric_df)
water_trainDS = CustomDataset(water_df)

electric_trainDL = DataLoader(electric_trainDS, batch_size = BATCH_SIZE)
water_trainDL = DataLoader(water_trainDS, batch_size = BATCH_SIZE)

In [ ]:
input_size = 28
# hidden_dim = 8
latent_dim = 16
n_layers = 2
penalty = 1
threshold = 0.02


model = LSTMAutoEncoderModel(input_size = input_size, latent_dim = latent_dim, n_layers = n_layers)

optimizer = optim.Adam(model.parameters(), lr = LR)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 10, verbose = True)


c:\Users\KDP-2\anaconda3\envs\Project_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
summary(model, input_size = (28, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 1, 16], [-1, 2, 16]]               0
               GRU-2       [[-1, 28], [-1, 28]]               0
            Linear-3                   [-1, 28]             812
Total params: 812
Trainable params: 812
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [8]:
for a, b in water_trainDL:
    print(len(a), len(b))
    break

64 64


In [ ]:
encoder_loss = training(model, electric_trainDL,
                    optimizer, penalty, threshold, EPOCH, scheduler, DEVICE)

[1 / 10000]
- TRAIN LOSS : 1.0788606093568
tensor([1.4200, 1.4800, 1.6400, 1.5300, 1.4600, 1.4400, 1.6300, 1.5400, 1.4600,
        1.4700, 1.5900, 1.3700, 1.3300, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400,
        1.6400, 1.5100, 1.4600, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400, 1.6400,
        1.5100])
tensor([1.4159, 1.4399, 1.6666, 1.5920, 1.4378, 1.4351, 1.6259, 1.5464, 1.5205,
        1.5719, 1.5760, 1.3710, 1.3746, 1.3505, 1.4539, 1.6719, 1.6483, 1.4497,
        1.4623, 1.5375, 1.4297, 1.3543, 1.5570, 1.7565, 1.6848, 1.5037, 1.4958,
        1.5806], grad_fn=<SelectBackward0>)
[2 / 10000]
- TRAIN LOSS : 1.0789204407397963
tensor([1.4200, 1.4800, 1.6400, 1.5300, 1.4600, 1.4400, 1.6300, 1.5400, 1.4600,
        1.4700, 1.5900, 1.3700, 1.3300, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400,
        1.6400, 1.5100, 1.4600, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400, 1.6400,
        1.5100])
tensor([1.4159, 1.4399, 1.6666, 1.5920, 1.4378, 1.4351, 1.6259, 1.5464, 1.5205,
        1.5719, 1.5760, 1.3710, 1

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(encoder_loss)
plt.show()